In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/class/lecture01_11/dacon_dataset

/content/drive/MyDrive/class/lecture01_11/dacon_dataset


In [2]:
img_size = 224
batch_size = 32

In [ ]:
#!unzip open.zip

In [4]:
import pandas as pd
import numpy as np 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import EfficientNetV2M, EfficientNetV2L, EfficientNetV2B3
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adamax, Adam
from tqdm import tqdm
import cv2
import numpy as np
from keras import Input

In [7]:
 train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32989,TRAIN_32989,./train/TRAIN_32989.jpg,1,0,0,0,1,0,1,1,1,1
32990,TRAIN_32990,./train/TRAIN_32990.jpg,1,0,0,0,0,1,1,1,1,1
32991,TRAIN_32991,./train/TRAIN_32991.jpg,1,0,0,0,0,1,1,1,1,1
32992,TRAIN_32992,./train/TRAIN_32992.jpg,1,0,0,0,0,1,1,1,1,1


In [ ]:
test_df

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg
...,...,...
1455,TEST_01455,./test/TEST_01455.jpg
1456,TEST_01456,./test/TEST_01456.jpg
1457,TEST_01457,./test/TEST_01457.jpg
1458,TEST_01458,./test/TEST_01458.jpg


In [8]:
df =  train_df.sample(frac=1)

In [9]:
train_len = int(len(df) * 0.8)

In [10]:
train = df[:train_len].reset_index(drop=True)
valid = df[train_len:].reset_index(drop=True)

In [11]:
path_train = (train['img_path'])
path_valid = (valid['img_path'])
path_test  = (test_df['img_path'])

In [12]:
import tensorflow as tf

### 인코딩 정의 함수

In [13]:
# 프로토콜 버퍼 인코딩, 디코딩 하는 함수
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
writer_image = tf.io.TFRecordWriter('image_train.tfr')
for i_, path_ in tqdm(enumerate(path_train)):

    src = cv2.imread(path_)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    dst = cv2.resize(dst, (224, 224), interpolation=cv2.INTER_CUBIC)
    bimage = dst.tobytes()
    
    classes = np.array(train.loc[i_, 'A':'J'], dtype=np.uint8).tobytes()
    
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': _bytes_feature(bimage),
                'classes': _bytes_feature(classes)
            }
            )
        )
    
    writer_image.write(example.SerializeToString())
    
writer_image.close()

26395it [02:24, 182.83it/s]


In [ ]:
writer_image_valid = tf.io.TFRecordWriter('image_valid.tfr')
for i_, path_ in tqdm(enumerate(path_valid)):

    src = cv2.imread(path_)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    dst = cv2.resize(dst, (224, 224), interpolation=cv2.INTER_CUBIC)    
    bimage = dst.tobytes()
    
    classes = np.array(valid.loc[i_, 'A':'J'], dtype=np.uint8).tobytes()
    
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': _bytes_feature(bimage),
                'classes': _bytes_feature(classes)            
            }
            )
        )
    
    writer_image_valid.write(example.SerializeToString())
    
writer_image_valid.close()

6599it [00:41, 159.75it/s]


In [ ]:
writer_image_test = tf.io.TFRecordWriter('image_test.tfr')

for i_, path_ in tqdm(enumerate(path_test)):

    src = cv2.imread(path_)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    dst = cv2.resize(dst, (224, 224), interpolation=cv2.INTER_CUBIC)
    
    bimage = dst.tobytes()
    
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': _bytes_feature(bimage)
            }
            )
        )
    
    writer_image_test.write(example.SerializeToString())
    
writer_image_test.close()

1460it [00:10, 142.00it/s]


In [14]:
## tfrecord file을 data로 parsing해주는 function
def _parse_function(tfrecord_serialized):
    features={'image': tf.io.FixedLenFeature([], tf.string),
              'classes': tf.io.FixedLenFeature([], tf.string)
             }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [img_size, img_size, 3])
    # image = tf.cast(image, tf.float32)/255. 

    classes = tf.io.decode_raw(parsed_features['classes'], tf.uint8)    
    classes = tf.squeeze(classes)

    return image, classes

def _parse_function2(tfrecord_serialized):
    features={'image': tf.io.FixedLenFeature([], tf.string)             }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [img_size, img_size, 3])
    # image = tf.cast(image, tf.float32)/255. 

    return image

In [15]:
## train dataset 만들기
train_dataset = tf.data.TFRecordDataset('image_train.tfr')
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(2000).prefetch(tf.data.experimental.AUTOTUNE).batch(32)

In [16]:
val_dataset = tf.data.TFRecordDataset('image_valid.tfr')
val_dataset = val_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE).batch(32)

In [17]:
test_dataset = tf.data.TFRecordDataset('image_test.tfr')
test_dataset = test_dataset.map(_parse_function2, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32)

In [ ]:
# !pip install keras_efficientnets

In [18]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
# from keras.applications.efficientnet_v2 import EfficientNetV2L

In [19]:
es = EarlyStopping(monitor = "val_loss", min_delta = 0, patience = 10,restore_best_weights=True)
checkpointer = ModelCheckpoint("model2.10epochs.hdf5", verbose = 1, save_best_only = True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-05)

In [20]:
base_model = EfficientNetV2M(weights = "imagenet", 
                   include_top=False,
                   input_shape = (224,224,3), pooling = "avg")

214201816/214201816 [==============================] - 10s 0us/step


In [21]:
for layer in base_model.layers :
  layer.trainable = False

In [22]:
last_output = base_model.output
batch = BatchNormalization()(last_output)
dens = Dense(1024, activation='elu')(batch)
dens = Dropout(0.4, seed = 123)(dens)
dens = Dense(100, activation='elu')(dens)
dens = Dense(32, activation='elu')(dens)
output = Dense(10, activation = "sigmoid")(dens)

new_model = Model(inputs = base_model.input, outputs = output)

In [ ]:
# new_model.load_weights("model.125epochs.hdf5")

In [23]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 24  648         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [24]:
new_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=Adamax(learning_rate = 1e-4), 
              metrics=['binary_accuracy']) 

In [ ]:
hist  = new_model.fit(train_dataset,
                  validation_data=val_dataset,    
                  verbose =1,
                  epochs=50, 
                  callbacks=[es, reduce_lr,checkpointer])

Epoch 1/50
    825/Unknown - 205s 212ms/step - loss: 0.6267 - binary_accuracy: 0.6584
Epoch 1: val_loss improved from inf to 0.51208, saving model to model2.10epochs.hdf5
825/825 [==============================] - 270s 290ms/step - loss: 0.6267 - binary_accuracy: 0.6584 - val_loss: 0.5121 - val_binary_accuracy: 0.7361 - lr: 1.0000e-04
Epoch 2/50
825/825 [==============================] - ETA: 0s - loss: 0.5547 - binary_accuracy: 0.7075
Epoch 2: val_loss improved from 0.51208 to 0.47932, saving model to model2.10epochs.hdf5
825/825 [==============================] - 220s 266ms/step - loss: 0.5547 - binary_accuracy: 0.7075 - val_loss: 0.4793 - val_binary_accuracy: 0.7588 - lr: 1.0000e-04
Epoch 3/50
825/825 [==============================] - ETA: 0s - loss: 0.5256 - binary_accuracy: 0.7271
Epoch 3: val_loss improved from 0.47932 to 0.45776, saving model to model2.10epochs.hdf5
825/825 [==============================] - 220s 266ms/step - loss: 0.5256 - binary_accuracy: 0.7271 - val_loss: 0

In [ ]:
pred_ = new_model.predict(test_dataset)

46/46 [==============================] - 14s 218ms/step


In [ ]:
df_subm = pd.read_csv('./sample_submission.csv')
df_subm.iloc[:, 1:] = ((pred_>0.5) + 0)
df_subm.to_csv('./subm2.csv', index=False)

In [ ]:
df_subm

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,1,1,0,0,0,0,1,0,1,0
1,TEST_00001,1,1,0,0,1,1,1,0,0,0
2,TEST_00002,1,1,0,0,1,1,0,1,0,1
3,TEST_00003,1,1,0,0,0,1,0,1,1,0
4,TEST_00004,0,1,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1455,TEST_01455,1,1,1,1,0,0,1,1,0,0
1456,TEST_01456,1,0,1,0,0,1,1,0,1,0
1457,TEST_01457,0,0,1,0,0,0,0,1,0,1
1458,TEST_01458,1,0,0,1,1,1,0,0,1,1
